# Design of AI systems

## Assignment 6, Eric Johansson & Max Sonnelid

# Monte Carlo Tree Search

In [1]:
import numpy as np
from collections import defaultdict

In [2]:
class MonteCarloTreeSearchNode:

    def __init__(self, state, parent):
        self.state = state
        self.results = defaultdict(int)
  
        self.V = 0
        self.untried_actions = None
        self.children = []
        self.parent = parent


    def checkUntriedActions (self):
        if self.untried_actions == None:
            self.untried_actions = self.state.possible_child_states() 
        return self.untried_actions

    def expand(self):
        new_state = self.untried_actions.pop()
        child = MonteCarloTreeSearchNode(new_state, parent = self) # try remove parent = 
        self.children.append(child)
        return child

    def rollout(self):
        current_rollout_state = self.state
        while not (current_rollout_state.check_win_3() in [-1,1] ): # Create this method in node class
            possible_actions = current_rollout_state.possible_child_states()
            if (len(possible_actions) == 0):        ## Fix this later!
                return current_rollout_state.check_win_3()
            move = self.rollout_policy(possible_actions)
            current_rollout_state = move
        
        return current_rollout_state.check_win_3() # Should return either -1 or 1 depending of who won the game
    
    def backpropagate(self, result):

        self.V += 1

        self.results[result] += 1
        if self.parent is not None:
            self.parent.backpropagate(result)

    def is_terminal_node(self):

        return self.state.check_win_3() in [-1,1]
    

    def is_fully_expanded(self):

        return (len(self.checkUntriedActions())==0)

    def best_child(self, c_param=1.0): ## Tagen från git
        visits = [
            c.V for c in self.children
        ]
        choices_weights = [
            (c.q() / c.V) + c_param * np.sqrt((2 * np.log(self.V) / c.V))
            for c in self.children
        ]

        return self.children[np.argmax(choices_weights)] 
    def best_child_2(self, c_param=1.0): ## Tagen från git
        visits = [
            c.V for c in self.children
        ]

        return self.children[np.argmax(visits)]


    def q(self):
        return self.results[self.state.current_player] ## Add playersTurn to board (either -1 or 1)


    def rollout_policy(self, possible_actions):
        return possible_actions[np.random.randint(len(possible_actions))]


In [3]:
class MonteCarloTreeSearch:
    def __init__(self, MCTSNode):
        self.parent = MCTSNode
    
    def best_action(self, simulations_number): # From Github
        for _ in range(0, simulations_number):            
            v = self.tree_policy()

            reward = v.rollout()
            v.backpropagate(reward)

        return self.parent.best_child_2(c_param=1.0)

    def tree_policy(self):
        current_node = self.parent
        while not current_node.is_terminal_node():
            if not current_node.is_fully_expanded():
   
                return current_node.expand()
            else:

                current_node = current_node.best_child()
        return current_node

    


In [4]:
from state import State

In [5]:
def play_game():
    matchState = State()

    for i in range(0,10):
        print(f"i = {i}")

        if (i == 0):
            matchState.print_state()
            print("")
    
        print("The current player is: ", matchState.current_player)
        print("")

        X = int(input())
        Y = int(input())
        matchState.draw(X,Y, matchState.current_player)
        matchState.print_state()

        print("")

        if (matchState.check_win_3() in [-1, 1]):
            print("Congratulations! Player ", matchState.check_win_3(), " has won.")
            return
        
        print("The current player is: ", 1, " (Booten Anna)")
        print("")

        matchStateNew = State()
        matchStateNew = matchState.deep_copy()
        root = MonteCarloTreeSearchNode(matchStateNew, parent = None)
        MCTS = MonteCarloTreeSearch(root)

        newState = MCTS.best_action(20).state
        
        matchState = newState.deep_copy()
        matchState.print_state()

        print("")

        
        if (matchState.check_win_3() in [-1, 1]):
            print("Congratulations! Player ", matchState.check_win_3(), " has won.")
            return

In [6]:
play_game()

i = 0
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

The current player is:  -1

[0, 0, 0]
[0, -1, 0]
[0, 0, 0]

The current player is:  1  (Booten Anna)

[0, 0, 0]
[0, -1, 0]
[0, 0, 1]

i = 1
The current player is:  -1

[0, 0, -1]
[0, -1, 0]
[0, 0, 1]

The current player is:  1  (Booten Anna)

[0, 0, -1]
[0, -1, 0]
[1, 0, 1]

i = 2
The current player is:  -1

[0, 0, -1]
[-1, -1, 0]
[1, 0, 1]

The current player is:  1  (Booten Anna)

[0, 0, -1]
[-1, -1, 0]
[1, 1, 1]

Congratulations! Player  1  has won.


In [7]:
def play_game_two_bots():
    matchState = State()

    for i in range(0,10):
       # print(f"i = {i}")

        if (i == 0):
            matchState.print_state()
            print("")
    
        print("The current player is: ", 1, " (Booten Anna)")
        print("")

        matchStatePlayerOne = State()
        matchStatePlayerOne = matchState.deep_copy()
        rootP1 = MonteCarloTreeSearchNode(matchStatePlayerOne, parent = None)
        MCTSP1 = MonteCarloTreeSearch(rootP1)

        matchStatePlayerOne = MCTSP1.best_action(20).state
        
        matchState = matchStatePlayerOne.deep_copy()
        matchState.print_state()

        print("")

        if (matchState.check_win_3() in [-1, 1]):
            print("Congratulations! Player ", matchState.check_win_3(), " has won.")
            return
        
        print("The current player is: ", -1, " (Booten Karl-Gustav)")
        print("")

        matchStateNew = State()
        matchStateNew = matchState.deep_copy()
        matchStateNew.current_player = 1
        root = MonteCarloTreeSearchNode(matchStateNew, parent = None)
        MCTS = MonteCarloTreeSearch(root)

        newState = MCTS.best_action(200).state
        
        matchState = newState.deep_copy()
        matchState.print_state()

        print("")

        
        if (matchState.check_win_3() in [-1, 1]):
            print("Congratulations! Player ", matchState.check_win_3(), " has won.")
            return

In [8]:
play_game_two_bots()

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

The current player is:  1  (Booten Anna)

[0, 0, 0]
[0, 0, 0]
[0, 0, 1]

The current player is:  -1  (Booten Karl-Gustav)

[0, 0, 0]
[0, -1, 0]
[0, 0, 1]

The current player is:  1  (Booten Anna)

[0, 0, 0]
[0, -1, 1]
[0, 0, 1]

The current player is:  -1  (Booten Karl-Gustav)

[0, -1, 0]
[0, -1, 1]
[0, 0, 1]

The current player is:  1  (Booten Anna)

[0, -1, 1]
[0, -1, 1]
[0, 0, 1]

Congratulations! Player  1  has won.
